# ProjectQ First Program


This exercise is based on the ProjectQ compiler tutorial. See https://github.com/ProjectQ-Framework/ProjectQ/blob/develop/examples/compiler_tutorial.ipynb for the original version.

Please check out [ProjectQ paper](http://arxiv.org/abs/1612.08091) for an introduction to the basic concepts behind this compiler.

This exercise will create the program to make the [superdense coding] (https://en.wikipedia.org/wiki/Superdense_coding).

## Load the modules

* projectq, includes main functionalities
* projectq.backends, includes the backends for the execution of the program. Initially, you will load the CommandPrinter, which prints the final gate sequence generated by the compilers.
* projectq.operation, includes the main defined operations, as common quantum gates (H,X,etc), or quantum full subroutines, as QFT.


In [1]:
import projectq
from projectq.backends import Simulator
from projectq.ops import CNOT, X, Y, Z, H, Measure,All

**The first step is to create an Engine. The sintax to create this object is using [MainEngine](http://projectq.readthedocs.io/en/latest/projectq.cengines.html#projectq.cengines.MainEngine):**
***
MainEngine(backend, engine_list, setups, verbose)
***

**In this case, the selected backend is [Simulator](http://projectq.readthedocs.io/en/latest/projectq.backends.html#projectq.backends.Simulator) which will simulate the final sequence of gates.**


In [2]:
# create the compiler and specify the backend:
eng = projectq.MainEngine(backend=Simulator())

**On this Engine, you must first allocate space for the qubits. You will allocate a register with two qubits.**

In [3]:
qureg = eng.allocate_qureg(2)

**First, you (Bob) must create the [Bell's state](https://en.wikipedia.org/wiki/Bell_state):**

$$|\psi\rangle = \frac{1}{\sqrt{2}} (|00\rangle+|11\rangle)$$

**To do it, apply a Hadamard gate (H) to the first qubit and, afterwards, a CNOT gate on qubit 1 using the qubit 0 as control bit.**

qubit 1 = qureg[1]

qubit 0 = qureg[0]

**To apply operations, ProjectQ uses the sintax:**

***
Operation | registers
***

**In the case of CNOT, the first qubit is the control qubit, the second the controlled qubit.**


In [4]:
H | qureg[0]
CNOT | (qureg[0],qureg[1])

In ProjectQ, nothing is computed until you flush the set of gates. At anytime, because you are using the simulator, you can get the state of the Quantum Register using the cheat backend operation. The first part shows how the qubits have been mapped and the second the current quantum state.  

In [5]:
eng.flush()
eng.backend.cheat()

({0: 0, 1: 1}, [(0.7071067811865475+0j), 0j, 0j, (0.7071067811865475+0j)])

**Now, after you (Bob) sent the qubit 1 to Alice, she applies one gate to transfer the information. The agreed protocol is:**

* 00, I
* 01, X
* 10, Z
* 11, Y

**Select one option for Alice!**


In [6]:
Y| qureg[1]

**Now, Alice sends her qubit to Bob, who uncomputes the entanglement (apply the inverse gates in reversed order. CNOT and H are their own inverse)**

In [7]:
CNOT | (qureg[0],qureg[1])
H | qureg[0]

**And, now, measure the results. In ProjectQ, to get the results, first you must flush the program content, so compilers and backends make their work. In this case, the Simulator.**

In [8]:
All(Measure) | qureg
eng.flush()
print("Message from Alice: {}{}".format(int(qureg[0]),int(qureg[1])))


Message from Alice: 11




**You can explore the sequence of engines that have been applied before the Simulator.**




In [9]:
engines=eng
while engines.next_engine!=None:
    print("Engine {}".format(engines.next_engine.__class__.__name__))
    engines=engines.next_engine

Engine TagRemover
Engine LocalOptimizer
Engine AutoReplacer
Engine TagRemover
Engine LocalOptimizer
Engine Simulator


# Congratulations!!! You have made you first Quantum Program with ProjectQ